In [3]:
import requests
import pandas as pd
import os
# from ./configs import *
import json

In [4]:
league_id = '169700'  # <- hit study
general_endpoint = "https://fantasy.premierleague.com/api/bootstrap-static/"
league_endpoint = f'https://fantasy.premierleague.com/api/leagues-classic/{league_id}/standings'
fixture_endpoint = 'https://fantasy.premierleague.com/api/fixtures/'

In [5]:
with requests.Session() as session:
    general_response = session.get(general_endpoint).json()
    league_response = session.get(league_endpoint).json()
    fixture_response = session.get(fixture_endpoint).json()

In [6]:
league_response

{'new_entries': {'has_next': False,
  'page': 1,
  'results': [{'entry': 4984487,
    'entry_name': 'Briyani Bois United',
    'joined_time': '2023-08-08T07:55:23.832900Z',
    'player_first_name': 'Tijil',
    'player_last_name': 'Jugadoe'},
   {'entry': 4719647,
    'entry_name': 'Flekken The Bean',
    'joined_time': '2023-08-07T13:10:49.556973Z',
    'player_first_name': 'Jason',
    'player_last_name': 'Perry'},
   {'entry': 2808378,
    'entry_name': 'NoHaaland',
    'joined_time': '2023-07-27T06:38:32.332976Z',
    'player_first_name': 'Connor',
    'player_last_name': 'McDonald'},
   {'entry': 364408,
    'entry_name': 'Big Mac Allister',
    'joined_time': '2023-07-24T09:07:26.704181Z',
    'player_first_name': 'Alex',
    'player_last_name': 'Wietzorrek'},
   {'entry': 2110155,
    'entry_name': 'BigMac Allister Meal',
    'joined_time': '2023-07-18T07:29:32.727545Z',
    'player_first_name': 'Alex',
    'player_last_name': 'Wietzorrek'},
   {'entry': 262206,
    'entry_name'

In [26]:
fixture_ids = pd.DataFrame.from_records(fixture_response, columns =['event', 'team_h', 'team_a'])

In [27]:
fixture_ids.head()

,event,team_h,team_a
0,1,6,13
1,1,1,16
2,1,3,19
3,1,5,12
4,1,9,10


In [28]:
gameweek_data = pd.DataFrame.from_records(general_response['events'], columns=['id', "deadline_time",'is_current', 'most_captained', 'most_vice_captained', 'top_element']).fillna(0).astype({"most_captained": int, "most_vice_captained": int, "top_element": int}).set_index('id')
gameweek_data.iloc[0,1] = True #<-- manually sets current week, remove once fpl is live
current_week = gameweek_data.index[gameweek_data['is_current']].item()

In [29]:
gameweek_data.head()

,deadline_time,is_current,most_captained,most_vice_captained,top_element
id,,,,,
1,2023-08-11T17:30:00Z,True,0,0,0
2,2023-08-19T12:30:00Z,False,0,0,0
3,2023-08-26T12:30:00Z,False,0,0,0
4,2023-09-02T12:30:00Z,False,0,0,0
5,2023-09-16T12:30:00Z,False,0,0,0


In [30]:
current_fixtures = fixture_ids[fixture_ids['event'] == current_week]
current_fixtures

,event,team_h,team_a
0,1,6,13
1,1,1,16
2,1,3,19
3,1,5,12
4,1,9,10
5,1,17,8
6,1,15,2
7,1,4,18
8,1,7,11
9,1,14,20


In [31]:
current_fixtures = current_fixtures.iloc[:,[1,2]]
current_fixtures

,team_h,team_a
0,6,13
1,1,16
2,3,19
3,5,12
4,9,10
5,17,8
6,15,2
7,4,18
8,7,11
9,14,20


In [32]:
current_fixtures.rename(columns={'team_h' : 'Home'}, inplace = True)
current_fixtures

/var/folders/c7/dcyx4ss130ldp4lp9ph0v16c0000gn/T/ipykernel_54870/662243259.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  current_fixtures.rename(columns={'team_h' : 'Home'}, inplace = True)


,Home,team_a
0,6,13
1,1,16
2,3,19
3,5,12
4,9,10
5,17,8
6,15,2
7,4,18
8,7,11
9,14,20
